# Learning seq2seq
The point of this notebook is to understand and implement a simple sequence-to-sequence (seq2seq) model. I'll generally be following [this tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html) on Keras' site.

### Imports

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

import numpy as np

### Model building
The general process is as follows:
- Split the model into an encoder and a decoder. 
- Encoder:
    - The encoder takes the input sequence as its input and outputs its internal state (both cell and hidden states, but only at the final timestep)
- Decoder:
    - The decoder uses as initial states the output states from the encoder
    - It uses as input the *target* sequence and is trained to predict the next timestep in the target sequence (again)
    
Note then that the entire purpose of the encoder is to generate a vector space representation of the input in terms of the hidden and cell states of the encoder. That is, the encoder serves to compress the information in the input sequencer.

The role of the decoder is to use this compression as information to decode a target sequence. Note that we *do not* use the input sequence in the decoder. This is because all information from the target sequence is (in theory) already held in the hidden states from the encoder. Thus the role of the decoder is to use that information to work directly with the target sequence. 

Let's now build the model.

In [204]:
def my_model(input_seq_length, vocab_size, batch_size, num_units_enc, target_seq_length, num_units_dec=None):
    if num_units_dec is None:
        num_units_dec = num_units_enc
        
    encoder_input = Input(shape=(input_seq_length, vocab_size), batch_size=batch_size, name='Encoder_Input')
    encoder_lstm = LSTM(units=num_units_enc, return_state=True, name='Encoder_LSTM')
    encoder_lstm_output = encoder_lstm(encoder_input)
    encoder_states = encoder_lstm_output[1:]
        
    decoder_input = Input(shape=(target_seq_length, vocab_size), batch_size=batch_size, name='Decoder_Input')
    # todo: Why is `return_sequences` needed? It relates to the input size of the final dense layer.
    decoder_lstm = LSTM(units=num_units_dec, return_sequences=True, name='Decoder_LSTM')
    decoder_lstm_output = decoder_lstm(decoder_input, initial_state=encoder_states) 
    decoder_dense = Dense(vocab_size, activation='softmax', name='Decoder_Dense')
    decoder_output = decoder_dense(decoder_lstm_output)
    
    model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)
    return model

### Data generation
We'll do the simple two-digit addition problem, since data is easy to generate from scratch. So the input sequence will be a string (one-hotted later) of digits and '+', and the output will be the sum (also as a string). We will include an end-of-string token for both as well. Like in the notebook we can either encode them as lists of integers and then let an embedding layer handle representing them as vectors, or one-hot everything. For simplicity (and to see what effects the embedding layer has later on) we'll one-hot for now.

In [126]:
char_to_int = {str(n): n for n in range(10)}
char_to_int[' '] = 10
char_to_int['+'] = 11
char_to_int['\n'] = 12

int_to_char = {v: k for k, v in char_to_int.items()}

In [127]:
def one_hot(n, dim):
    # One-hots a positive integer n
    one_hot_n = np.zeros(dim)
    one_hot_n[n] = 1
    return one_hot_n
    
def undo_one_hot(v):
    return np.argmax(v)

def one_hot_matrix(M, vocab_size):
    n_samples, seq_length = M.shape
    M_oh = np.array([one_hot(r, vocab_size) for r in np.array(M).flatten()]).reshape((n_samples, seq_length, vocab_size))
    return np.squeeze(M_oh) # In case this is a target vector, we don't want to include an unnecessary axis

In [247]:
def generate_sample(num_terms=2, digit_length=2, max_target_digits=3, int_encoder=None, reverse=False):
    x = []
    for _ in range(num_terms):
        x.append(np.random.randint(10**digit_length))
        
    y = np.sum(x)
    
    x_str = '+'.join(str(n) for n in x)
    y_str = str(y)
    
    # Pad x so that is always has the same length. It should be of length digit_length for each digit, plus num_terms - 1 "plus" signs
    x_str = x_str.rjust(num_terms * digit_length + num_terms - 1)
    y_str = y_str.rjust(max_target_digits) # todo: Fix to be a calculated value
    
    if reverse:
        x_str = x_str[::-1]

    x_str += '\n'
    y_str += '\n'
    
    x_list = list(x_str)
    y_list = list(y_str)
    
    if int_encoder is not None:
        assert isinstance(int_encoder, dict), 'int_encoder must be a dictionary mapping characters to integers'
        x_list = [int_encoder[c] for c in x_list]
        y_list = [int_encoder[c] for c in y_list]
        
    return x_list, y_list

In [248]:
def generate_samples(n_samples, num_terms=2, digit_length=2, max_target_digits=3, int_encoder=None, one_hot=False, reverse=False):
    X = []
    y = []
    for _ in range(n_samples):
        x_sample, y_sample = generate_sample(num_terms, digit_length, max_target_digits, int_encoder, reverse)
        X.append(x_sample)
        y.append(y_sample)
        
    X = np.array(X)
    y = np.array(y)
    
    if one_hot:
        X = one_hot_matrix(X, len(int_encoder))
        y = one_hot_matrix(y, len(int_encoder))
    
    return X, y

In [130]:
generate_sample(append_token=True)

(['7', '3', '+', '6', '7', '\n'], ['1', '4', '0', '\n'])

In [131]:
generate_sample(append_token=True, int_encoder=char_to_int)

([3, 6, 11, 3, 1, 12], [10, 6, 7, 12])

In [132]:
X_oh, y_oh = generate_samples(n_samples=10, append_token=True, int_encoder=char_to_int, one_hot=True)

In [133]:
X_oh.shape, y_oh.shape

((10, 6, 13), (10, 4, 13))

### Test out the model
Let's run some data through our model to make sure everything works as intended.

In [134]:
NUM_TERMS = 2
DIGIT_LENGTH = 2
SEQ_LENGTH = NUM_TERMS * DIGIT_LENGTH + NUM_TERMS

model = my_model(input_seq_length=SEQ_LENGTH, vocab_size=len(char_to_int), batch_size=1, num_units_enc=1, target_seq_length=4)

In [135]:
def example_prediction(model, X, y):
    X_singleton = X_oh[0].reshape(1, *X_oh[0].shape)
    y_singleton = y_oh[0].reshape(1, *y_oh[0].shape)

    output = model.predict([X_singleton, y_singleton], batch_size=1)
    return output

def sample_from_softmax(output):
    squeezed_output = np.squeeze(output)
    sampled_output = np.random.choice(np.arange(len(squeezed_output)), p=squeezed_output)
    return sampled_output

def predict_one(model, X, y):
    model_output = example_prediction(model, X, y)
    sampled_pred = sample_from_softmax(model_output)
    # todo: Make this output nicer to get a better idea what's actually being predicted/used
    print(f'X = {X}')
    print(f'y = {y}')
    print(f'Pred = {sampled_pred}')

In [136]:
example_prediction(model, X, y)

array([[0.07845037, 0.07061684, 0.07993645, 0.08047733, 0.07797949,
        0.07469489, 0.07416169, 0.07028104, 0.06670289, 0.08318146,
        0.08559087, 0.07626706, 0.08165962]], dtype=float32)

In [137]:
predict_one(model, X, y)

X = [[ 8  3 11  9  4 12]
 [ 1  8 11  1  6 12]
 [ 3  6 11  1  3 12]
 [ 9  9 11  5  4 12]
 [10  1 11  3  1 12]
 [ 1  7 11  8  4 12]
 [ 6  1 11  9  8 12]
 [ 4  9 11  3  3 12]
 [10  0 11  4  2 12]
 [10  4 11  9  4 12]]
y = [[ 1  7  7 12]
 [10  3  4 12]
 [10  4  9 12]
 [ 1  5  3 12]
 [10  3  2 12]
 [ 1  0  1 12]
 [ 1  5  9 12]
 [10  8  2 12]
 [10  4  2 12]
 [10  9  8 12]]
Pred = 9


### Train the model

In [179]:
X_oh, y_oh = generate_samples(n_samples=10**4, append_token=True, int_encoder=char_to_int, one_hot=True)

In [201]:
model = my_model(input_seq_length=SEQ_LENGTH, vocab_size=len(char_to_int), batch_size=10, num_units_enc=1, target_seq_length=3)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [182]:
input_target = y_oh[:,:-1,:]
output_target = y_oh[:,1:,:]
model.fit([X_oh, input_target], output_target, epochs=30, validation_split=0.3)

Train on 7000 samples, validate on 3000 samples
Epoch 1/30
7000/7000 [==============================] - 13s 2ms/sample - loss: 2.3529 - accuracy: 0.3184 - val_loss: 2.1386 - val_accuracy: 0.3333
Epoch 2/30
7000/7000 [==============================] - 8s 1ms/sample - loss: 1.9947 - accuracy: 0.3459 - val_loss: 1.8739 - val_accuracy: 0.3671
Epoch 3/30
7000/7000 [==============================] - 8s 1ms/sample - loss: 1.7964 - accuracy: 0.3645 - val_loss: 1.7337 - val_accuracy: 0.3693
Epoch 4/30
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.6945 - accuracy: 0.3673 - val_loss: 1.6621 - val_accuracy: 0.3737
Epoch 5/30
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.6395 - accuracy: 0.3944 - val_loss: 1.6206 - val_accuracy: 0.4010
Epoch 6/30
7000/7000 [==============================] - 9s 1ms/sample - loss: 1.6071 - accuracy: 0.4024 - val_loss: 1.5961 - val_accuracy: 0.4003
Epoch 7/30
7000/7000 [==============================] - 8s 1ms/sample - los

So training for 30 epochs with a single unit in the LSTM gives ~43% validation accuracy. 

Let's do it again, but mirroring the parameters set by the [Addition RNN](https://keras.io/examples/addition_rnn/) script at Keras. They train using three digits with 5k samples with 10% validation for 200 epochs with a batch size of 128 and with 128 units in the LSTM and achieve 99% accuracy.

In [239]:
NUM_TERMS = 2
DIGIT_LENGTH = 3
SEQ_LENGTH = NUM_TERMS * DIGIT_LENGTH + NUM_TERMS

X_oh, y_oh = generate_samples(n_samples=5*10**3, \
                              digit_length=DIGIT_LENGTH, \
                              max_target_digits=4, \
                              append_token=True, \
                              int_encoder=char_to_int, \
                              one_hot=True)

In [240]:
model = my_model(input_seq_length=SEQ_LENGTH, vocab_size=len(char_to_int), batch_size=128, num_units_enc=128, target_seq_length=4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [242]:
input_target = y_oh[:,:-1,:]
output_target = y_oh[:,1:,:]
model.fit([X_oh, input_target], output_target, epochs=200)

Train on 5000 samples
Epoch 1/200
5000/5000 [==============================] - 7s 1ms/sample - loss: 2.2103 - accuracy: 0.2530
Epoch 2/200
5000/5000 [==============================] - 2s 470us/sample - loss: 1.7638 - accuracy: 0.3296
Epoch 3/200
5000/5000 [==============================] - 2s 496us/sample - loss: 1.7350 - accuracy: 0.3408
Epoch 4/200
5000/5000 [==============================] - 3s 502us/sample - loss: 1.7261 - accuracy: 0.3435
Epoch 5/200
5000/5000 [==============================] - 2s 497us/sample - loss: 1.7174 - accuracy: 0.3496
Epoch 6/200
5000/5000 [==============================] - 3s 506us/sample - loss: 1.7085 - accuracy: 0.3485
Epoch 7/200
5000/5000 [==============================] - 3s 518us/sample - loss: 1.7005 - accuracy: 0.3503
Epoch 8/200
5000/5000 [==============================] - 3s 508us/sample - loss: 1.6904 - accuracy: 0.3491
Epoch 9/200
5000/5000 [==============================] - 3s 509us/sample - loss: 1.6775 - accuracy: 0.3541
Epoch 10/200
5000

This seems promising. Let's generate a test set and see how it performs.

In [243]:
X_test, y_test = generate_samples(n_samples=1000, \
                              digit_length=DIGIT_LENGTH, \
                              max_target_digits=4, \
                              append_token=True, \
                              int_encoder=char_to_int, \
                              one_hot=True)

In [244]:
test_metrics = model.evaluate([X_test, y_test[:, :-1, :]], y_test[:, 1:, :], verbose=0)

In [245]:
print('\n'.join(f'{n} = {v:.4f}' for n, v in list(zip(model.metrics_names, test_metrics))))

loss = 0.9939
accuracy = 0.6507


So an improvement, but still well short of the 99% they claim.

Next, let's try adding in "reversing". They claim this gives a good boost.

In [249]:
NUM_TERMS = 2
DIGIT_LENGTH = 3
SEQ_LENGTH = NUM_TERMS * DIGIT_LENGTH + NUM_TERMS

X_oh, y_oh = generate_samples(n_samples=5*10**3, \
                              digit_length=DIGIT_LENGTH, \
                              max_target_digits=4, \
                              int_encoder=char_to_int, \
                              one_hot=True,
                              reverse=True)

In [250]:
model = my_model(input_seq_length=SEQ_LENGTH, vocab_size=len(char_to_int), batch_size=128, num_units_enc=128, target_seq_length=4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [251]:
input_target = y_oh[:,:-1,:]
output_target = y_oh[:,1:,:]
model.fit([X_oh, input_target], output_target, epochs=200)

Train on 5000 samples
Epoch 1/200
5000/5000 [==============================] - 7s 1ms/sample - loss: 2.1874 - accuracy: 0.2650
Epoch 2/200
5000/5000 [==============================] - 2s 464us/sample - loss: 1.7598 - accuracy: 0.3256
Epoch 3/200
5000/5000 [==============================] - 2s 492us/sample - loss: 1.7328 - accuracy: 0.3413
Epoch 4/200
5000/5000 [==============================] - 3s 503us/sample - loss: 1.7245 - accuracy: 0.3446
Epoch 5/200
5000/5000 [==============================] - 2s 494us/sample - loss: 1.7174 - accuracy: 0.3470
Epoch 6/200
5000/5000 [==============================] - 2s 499us/sample - loss: 1.7077 - accuracy: 0.3492
Epoch 7/200
5000/5000 [==============================] - 3s 504us/sample - loss: 1.6982 - accuracy: 0.3503
Epoch 8/200
5000/5000 [==============================] - 3s 516us/sample - loss: 1.6905 - accuracy: 0.3492
Epoch 9/200
5000/5000 [==============================] - 3s 517us/sample - loss: 1.6831 - accuracy: 0.3533
Epoch 10/200
5000

In [252]:
X_test, y_test = generate_samples(n_samples=1000, \
                              digit_length=DIGIT_LENGTH, \
                              max_target_digits=4, \
                              int_encoder=char_to_int, \
                              one_hot=True, \
                              reverse=True)

In [253]:
test_metrics = model.evaluate([X_test, y_test[:, :-1, :]], y_test[:, 1:, :], verbose=0)

In [254]:
print('\n'.join(f'{n} = {v:.4f}' for n, v in list(zip(model.metrics_names, test_metrics))))

loss = 0.1018
accuracy = 0.9737


Wow. So reversing the input increased our test accuracy from 65% to 97%, and resulted in a nearly 90% reduction in the loss. That's incredible. Reverse reverse reverse!